In [24]:
import os

# Manually set the environment variables
os.environ["VISION_KEY"] = "8JKzGpabbAjjakJdpgaiDpWUcemTiX3OtJDnauvjUlngn5HKpk9yJQQJ99BCAC5RqLJXJ3w3AAAFACOGfpJN"
os.environ["VISION_ENDPOINT"] = "https://georgiana.cognitiveservices.azure.com/"

# Verify if the variables are set
print("VISION_KEY:", os.environ.get("VISION_KEY"))
print("VISION_ENDPOINT:", os.environ.get("VISION_ENDPOINT"))


VISION_KEY: 8JKzGpabbAjjakJdpgaiDpWUcemTiX3OtJDnauvjUlngn5HKpk9yJQQJ99BCAC5RqLJXJ3w3AAAFACOGfpJN
VISION_ENDPOINT: https://georgiana.cognitiveservices.azure.com/


In [25]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
from PIL import Image
import sys
import time
'''
Authenticate
Authenticates your credentials and creates a client.
'''
subscription_key = os.environ["VISION_KEY"]
endpoint = os.environ["VISION_ENDPOINT"]
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
'''
END - Authenticate
'''

'\nEND - Authenticate\n'

# Exemplul

In [26]:
# img = open("test1.png", "rb")
img = open("test2.jpeg", "rb")
read_response = computervision_client.read_in_stream(
    image=img,
    mode="Printed",
    raw=True
)
# print(read_response.as_dict())

operation_id = read_response.headers['Operation-Location'].split('/')[-1]
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

# Print the detected text, line by line
result = []
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            result.append(line.text)

print()

Lucces in resolvarea
TEMELOR la
LABORA toarele de
Inteligenta Artificialà!


In [27]:
# get/define the ground truth
# groundTruth = ["Google Cloud", "Platform"]
groundTruth = ["Succes in rezolvarea", "tEMELOR la", "LABORAtoaree de", "Inteligenta Artificiala!"]

# compute the performance
noOfCorrectLines = sum(i == j for i, j in zip(result, groundTruth))
print(noOfCorrectLines)

0


# Ex 1. calitatea procesului de recunoastere a textului, atat la nivel de caracter, cat si la nivel de cuvant

# a.prin folosirea unei metrici de distanta sau

In [39]:
import cv2

# Incarc imaginea
image = cv2.imread('image.jpeg')

# Convertim imaginea în tonuri de gri
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Aplic un filtru de thresholding (binarizare)
_, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)

# Salvez imaginea preprocesata
cv2.imwrite('preprocessed_image.jpg', binary)

True

In [41]:
from PIL import Image
import pytesseract

# Configurez calea catre executabilul Tesseract (Windows)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Incarc imaginea preprocesata
image_path = 'preprocessed_image.jpg'
image = Image.open(image_path)

# Extrag textul folosind Tesseract
recognized_text = pytesseract.image_to_string(image, lang='eng')
print("Text recunoscut:", recognized_text)

Text recunoscut: Gecaus €
be doing


In [42]:
import Levenshtein

# Textul corect (ground truth)
ground_truth = "Because be doing"

# Calculez distanța Levenshtein
distance = Levenshtein.distance(ground_truth, recognized_text)

# Calculez CER
cer = distance / len(ground_truth)
print("Character Error Rate (CER):", cer)

Character Error Rate (CER): 0.3125


# b. prin folosirea mai multor metrici de distanta.

In [4]:
import Levenshtein

# Functie pentru calcularea CER
def calculate_cer(ground_truth, recognized_text):
    return Levenshtein.distance(ground_truth, recognized_text) / len(ground_truth)

# Functie pentru calcularea WER
def calculate_wer(ground_truth, recognized_text):
    ground_truth_words = ground_truth.split()
    recognized_words = recognized_text.split()
    return Levenshtein.distance(ground_truth_words, recognized_words) / len(ground_truth_words)

# Functie pentru calcularea distantei Hamming
def calculate_hamming(ground_truth, recognized_text):
    if len(ground_truth) != len(recognized_text):
        raise ValueError("Șirurile trebuie să aibă aceeași lungime pentru distanța Hamming")
    return Levenshtein.hamming(ground_truth, recognized_text)

# Textul corect (ground truth)
ground_truth = "Because be doing"

# Textul recunoscut
recognized_text = "Gecaus€ be doing"

# Calculam CER
cer = calculate_cer(ground_truth, recognized_text)

# Calculăm WER
wer = calculate_wer(ground_truth, recognized_text)

# Calculam distanta Hamming
try:
    hamming_dist = calculate_hamming(ground_truth, recognized_text)
except ValueError as e:
    hamming_dist = str(e)

# Afisam rezultatele
print("Character Error Rate (CER):", cer)
print("Word Error Rate (WER):", wer)
print("Distanta Hamming:", hamming_dist)

Character Error Rate (CER): 0.125
Word Error Rate (WER): 0.3333333333333333
Distanța Hamming: 2
